Amirmohammad Khosravi Esfezar
S.N.: 810198386

Doctor Strange Project:
find the best way for Dr.Strange to collect all potions in a temple and deliver them to target.

In [250]:
import copy
import time
import heapq
import bisect

Choosing test case, algorithm and alpha:

In [251]:
algorithms = {1: "BFS", 2: "IDS", 3: "ASTAR"}
algoType = 3
testNum = 3
alpha = 1.5
totalSeenStates = 0

TEST 1 

| Algorithm | Answer Distance | Seen States Count | Average Run Time
| --- | --- | --- | --- |
| BFS | 11 | 3818 | 106 ms | 
| IDS | 11 | 23406 | 573 ms | 
| A* | 11 | 2359 | 183 ms |
| Weighted A* | 11 | 1379 | 112 ms |

TEST 2

| Algorithm | Answer Distance | Seen States Count | Average Run Time
| --- | --- | --- | --- |
| BFS | 11 | 340900 | 10453 ms | 
| IDS | 11 | 871136 | 27895 ms | 
| A* | 11 | 16304 | 1307 ms |
| Weighted A* | 11 | 7032 | 743 ms |

TEST 3

| Algorithm | Answer Distance | Seen States Count | Average Run Time
| --- | --- | --- | --- |
| BFS | 19 | 541191 | 21216 ms | 
| IDS | 19 | 2255356 | 95689 ms | 
| A* | 19 | 62040 | 9987 ms |
| Weighted A* | 19 | 27258 | 4422 ms |

Reading the file: First we read data from file and save it in a 2D array of integers

In [252]:
def readFile(filePath):
    fileData = open(filePath, 'r')
    templeDataStr = []
    templeData = []
    templeDataStr = fileData.readlines()
    for i in templeDataStr:
        lineStr = i.split()
        lineInt = []
        for j in lineStr:
            lineInt.append(int(j))
        templeData.append(lineInt)
    return templeData

Problem Modeling:
We create two classes called Temple and State. 
Temple objects contain size of the Temple (number of rows and columns), the coordinates of blocked cells, coodinates of double drugs and potions. It also have a method called checkIfValid() which gets a coordinates and tells if doctor can go there or not.
State objects contain a list of all doctors coordinates, a Temple object, address of the parent State, state generation cost and a string which represent the whole State object in a compressed way. The hash string contains number of remaining potions and coordinates of all doctors, potions and double drugs. State also has two methods called __lt__() and checkIfGoulState(). __lt__() is used for min heap and chechIfGoulState() checks if all potions collected and all doctors are in the target cell or not, if yes then it will return True which means the state is goul state.

In [253]:
class Temple:
    def __init__(self, rows, cols):
        self.rowCounts = rows
        self.colCounts = cols
        self.blockedCells = []
        self.doubleDrugs = []
        self.potions = []
        
    def checkIfValid(self, x, y):
        if x < 0 or y < 0 or x >= self.rowCounts or y >= self.colCounts or [x, y] in self.blockedCells:
            return False
        return True

class State:
    def __init__(self, _doctors, _potions, _temple, _parent, cost):
        self.doctors = _doctors
        self.temple = _temple
        self.parent = _parent
        self.genCost = cost
        self.AStarCost = 0
        self.hashStr = str(len(self.temple.potions))
        for i in self.doctors:
            self.hashStr += str(i[0]) + str(i[1])
        for i in self.temple.potions:
            for j in i:
                self.hashStr += str(int(j))
        for i in self.temple.doubleDrugs:
            for j in i:
                self.hashStr += str(int(j))
        
    def checkDoctorArrive(self, x, y):
        if len(self.temple.potions) == 0:
            if x == self.temple.rowCounts - 1 and y == self.temple.colCounts - 1:
                return True
        return False
    
    def __lt__(self, other):
        return self.AStarCost < other.AStarCost
            
    def checkIfGoulState(self):
        if len(self.temple.potions) > 0:
            return False
        for i in self.doctors:
            if i[0] != self.temple.rowCounts - 1 or i[1] != self.temple.colCounts - 1:
                return False
        return True

The following function create initial state which contains all double drugs and potions and the only doctor is in the first cell (coordinates [0, 0]). The function gets a 2D array which contains temple's data.

In [254]:
def getInitialState(templeData):
    global totalSeenStates
    potions = []
    doubleDrugs = []
    po, dr = templeData[1][0], templeData[1][1]
    temple = Temple(templeData[0][0], templeData[0][1])
    potions = templeData[2:po + 2:]
    temple.doubleDrugs = templeData[po + 2:po + dr + 2:]
    temple.blockedCells = templeData[po + dr + 3:po + dr + 3 + templeData[po + dr + 2][0]:]
    temple.potions = potions.copy()
    initialState = State([[0, 0]], potions, temple, None, 0)
    return initialState

The following functions just print data of a state. we will use them at the end of the process to show the steps and the final state and also check if the answer is correct or not. The second function prints all the steps and lists movements of all doctors.

In [255]:
def printState(state):
    print("*" * 40)
    print("total seen states :", totalSeenStates)
    print("doctors :", state.doctors)
    print("potions :", len(state.temple.potions))
    print("blockedCells :", state.temple.blockedCells)
    print("doubleDrugs :", state.temple.doubleDrugs)
    print("cost :", state.genCost)
    print("hashStr : ", state.hashStr)
    print("A* cost :", state.AStarCost)
    if state.parent != None:
        printStateParents(state)
    print("*" * 40)

def printStateParents(state):
    print("#" * 40)
    while (state.parent != None):
        state = state.parent
        print(state.doctors)
    print("#" * 40)

The following function does the unknown BFS. First we make a queue called toVisit and push initial state in it. Then for each state we expand it and push all new states to toVisit. Every time we push one state, we save its hashed string into a set called visited to avoid visiting states more than once. For each new state we check if it is goul state or not, if it's goul state we return and print it.
The unknown BFS algorithm is faster than unknown IDS so BFS has less time complexity than IDS. Unknown BFS uses more memory while processing than unknown IDS so space complexity of BFS is more than IDS. both BFS and IDS are slower than A* algorithm and use more memory.

In [256]:
def doUnknownBfs(templeData):
    global totalSeenStates
    neighbors = [[1, 0], [0, 1], [-1, 0], [0, -1]]
    initialState = getInitialState(templeData)
    visited = {initialState.hashStr}
    toVisit = [initialState]
    if [0, 0] in initialState.temple.doubleDrugs:
        initialState.temple.doubleDrugs.remove([0, 0])
        initialState.doctors.append([initialState.temple.rowCounts - 1, 0])
    while len(toVisit) != 0:
        for i in toVisit[0].doctors:
            for j in neighbors:
                if toVisit[0].temple.checkIfValid(i[0] + j[0], i[1] + j[1]):
                    temple = copy.deepcopy(toVisit[0].temple)
                    doctors = copy.deepcopy(toVisit[0].doctors)
                    for k in doctors:
                        if k == i:
                            k[0] = i[0] + j[0]
                            k[1] = i[1] + j[1]
                            break
                    if [i[0] + j[0], i[1] + j[1]] in temple.potions:
                        temple.potions.remove([i[0] + j[0], i[1] + j[1]])
                    if [i[0] + j[0], i[1] + j[1]] in temple.doubleDrugs:
                        temple.doubleDrugs.remove([i[0] + j[0], i[1] + j[1]])
                        doctors.append([temple.rowCounts - 1, 0])
                    newState = State(doctors, [], temple, toVisit[0], toVisit[0].genCost + 1)
                    totalSeenStates += 1
                    if newState.checkIfGoulState():
                        return newState
                    if newState.hashStr not in visited:
                        toVisit.append(newState)
                        visited.add(newState.hashStr)
        toVisit.pop(0)

The following function does the unknown IDS. It does a DFS for all different depths.For each state we expand it and do a DFS on every new state. Every time we do DFS on one state, we save its hashed string into a set called visited to avoid visiting states more than once. For each new state we check if it is goul state or not, if it's goul state we return and print it.
The unknown IDS algorithm is slower than unknown IDS so BFS has less time complexity than IDS. Unknown BFS uses more memory while processing than unknown IDS so space complexity of BFS is more than IDS. both BFS and IDS are slower than A* algorithm and use more memory.

In [257]:
def ids(state, curDepth, maxDepth, visited):
    global totalSeenStates
    if curDepth >= maxDepth:
        return None
    if state.hashStr + str(state.genCost) in visited:
        return None
    visited.add(state.hashStr + str(state.genCost))
    neighbors = [[1, 0], [0, 1], [-1, 0], [0, -1]]
    for i in state.doctors:
        for j in neighbors:
            if state.temple.checkIfValid(i[0] + j[0], i[1] + j[1]):
                temple = copy.deepcopy(state.temple)
                doctors = copy.deepcopy(state.doctors)
                for k in doctors:
                    if k == i:
                        k[0] = i[0] + j[0]
                        k[1] = i[1] + j[1]
                        if state.checkDoctorArrive(k[0], k[1]):
                            doctors.remove(k)
                        break
                if [i[0] + j[0], i[1] + j[1]] in temple.potions:
                    temple.potions.remove([i[0] + j[0], i[1] + j[1]])
                if [i[0] + j[0], i[1] + j[1]] in temple.doubleDrugs:
                    temple.doubleDrugs.remove([i[0] + j[0], i[1] + j[1]])
                    doctors.append([temple.rowCounts - 1, 0])
                newState = State(doctors, [], temple, state, state.genCost + 1)
                totalSeenStates += 1
                if newState.checkIfGoulState():
                    return newState
                returned = ids(newState, curDepth + 1, maxDepth, visited)
                if returned != None:
                    return returned
    
def doUnknownIds(templeData):
    neighbors = [[1, 0], [0, 1], [-1, 0], [0, -1]]
    initialState = getInitialState(templeData)
    visited = set()
    if [0, 0] in initialState.temple.doubleDrugs:
        initialState.temple.doubleDrugs.remove([0, 0])
        initialState.doctors.append([initialState.temple.rowCounts - 1, 0])
    goulState = None
    i = 1
    while goulState == None:
        visited.clear()
        goulState = ids(initialState, 0, i, visited)
        i += 1
    return goulState

The following functions are for A* algorithm. The algorithm is similar to unknown BFS. We just push new state in a min heap so each time we will get the state with least heuristic cost to expand. the heuristic fucntion is the total of minimum distanse of doctors to target or a potions. the function is consistent..
Both BFS and IDS are slower than A* algorithm and use more memory. 

In [258]:
def evaluationF(state):
    aStarCost = 0
    for i in state.doctors:
        minPotion = 999999
        for j in state.temple.potions:
            distance = abs(i[0] - j[0]) + abs(i[1] - j[1])
            if distance < minPotion:
                minPotion = distance
        aStarCost += min((abs(state.temple.rowCounts - 1 - i[0]) + abs(state.temple.colCounts - 1 - i[1])), minPotion)
    aStarCost = alpha * aStarCost
    state.AStarCost = aStarCost + state.genCost
        
def doKnownAStar(templeData):
    global totalSeenStates
    neighbors = [[1, 0], [0, 1], [-1, 0], [0, -1]]
    initialState = getInitialState(templeData)
    visited = {initialState.hashStr}
    toVisit = [initialState]
    if [0, 0] in initialState.temple.doubleDrugs:
        initialState.temple.doubleDrugs.remove([0, 0])
        initialState.doctors.append([initialState.temple.rowCounts - 1, 0])
    evaluationF(initialState)
    while len(toVisit) != 0:
        toVisit0 = copy.deepcopy(toVisit[0])
        toVisit.pop(0)
        for i in toVisit0.doctors:
            for j in neighbors:
                if toVisit0.temple.checkIfValid(i[0] + j[0], i[1] + j[1]):
                    temple = copy.deepcopy(toVisit0.temple)
                    doctors = copy.deepcopy(toVisit0.doctors)
                    for k in doctors:
                        if k == i:
                            k[0] = i[0] + j[0]
                            k[1] = i[1] + j[1]
                            break
                    if [i[0] + j[0], i[1] + j[1]] in temple.potions:
                        temple.potions.remove([i[0] + j[0], i[1] + j[1]])
                    if [i[0] + j[0], i[1] + j[1]] in temple.doubleDrugs:
                        temple.doubleDrugs.remove([i[0] + j[0], i[1] + j[1]])
                        doctors.append([temple.rowCounts - 1, 0])
                    newState = State(doctors, [], temple, toVisit0, toVisit0.genCost + 1)
                    totalSeenStates += 1
                    evaluationF(newState)
                    if newState.checkIfGoulState():
                        return newState
                    if newState.hashStr not in visited:
                        bisect.insort(toVisit, newState)
                        visited.add(newState.hashStr)

The following function decide which algorithm 

In [259]:
def solve(algorithm, filePath):
    templeData = readFile(filePath)
    if algorithm == "BFS":
        return doUnknownBfs(templeData)
    if algorithm == "IDS":
        return doUnknownIds(templeData)
    if algorithm == "ASTAR":
        return doKnownAStar(templeData)

In [260]:
def main():
    startTime = time.time()
    printState(solve(algorithms[algoType], "Tests/test" + str(testNum) + ".in"))
    endTime = time.time()
    print("Time: ", '{:.5f}'.format((endTime - startTime) * 1000), " ms")

In [261]:
main()

****************************************
total seen states : 27258
doctors : [[5, 5], [5, 5]]
potions : 0
blockedCells : [[0, 2], [1, 1], [4, 1], [3, 0]]
doubleDrugs : [[4, 2]]
cost : 19
hashStr :  0555542
A* cost : 19.0
########################################
[[4, 5], [5, 5]]
[[3, 5], [5, 5]]
[[2, 5], [5, 5]]
[[1, 5], [5, 5]]
[[1, 4], [5, 5]]
[[1, 3], [5, 5]]
[[1, 2], [5, 5]]
[[2, 2], [5, 5]]
[[2, 2], [5, 4]]
[[2, 2], [5, 3]]
[[2, 2], [4, 3]]
[[2, 2], [5, 3]]
[[2, 2], [5, 2]]
[[2, 2], [5, 1]]
[[2, 2], [5, 0]]
[[2, 1]]
[[2, 0]]
[[1, 0]]
[[0, 0]]
########################################
****************************************
Time:  4678.44105  ms
